In [14]:
import chromadb
import torch
print(torch.cuda.is_available())  # Should be True if CUDA is available
print(torch.cuda.current_device())  # Print the index of the current GPU
print(torch.cuda.get_device_name(0))  # Print the name of the GPU

True
0
NVIDIA GeForce RTX 3070 Laptop GPU


In [15]:
import torch
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone as PL
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate

from loguru import logger as logging

from dotenv import load_dotenv
load_dotenv()

True

In [16]:
model_name = "BAAI/bge-m3"

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

In [18]:
news_df = pd.read_csv('../../lenta-ru-news.csv')
loader = DataFrameLoader(news_df, page_content_column='text')
documents = loader.load()
len(news_df)

800975

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 420, chunk_overlap = 380)
docs = text_splitter.split_documents(documents)
len(docs)

19296165

In [ ]:
def load_and_prepare_documents():
    news_df = pd.read_csv('../lenta-ru-news.csv')
    loader = DataFrameLoader(news_df, page_content_column='text')
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 420, chunk_overlap = 380)
    docs = text_splitter.split_documents(documents)
    return docs

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)
index_name = os.getenv('PINECONE_API_INDEX_NAME')
logging.info(f'Current index name: {index_name}')


if index_name not in pc.list_indexes().names():
    logging.info(f'Creating new indexes')
    docs = load_and_prepare_documents()
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric='cosine',
        spec=ServerlessSpec(
            cloud=os.getenv('PINECONE_API_CLOUD'),
            region=os.getenv('PINECONE_API_ENV')
        )
    )
    docsearch = PL.from_documents(docs, embedding_function, index_name)
else:
    logging.info(f'Importing indexes from Pinecone index "{index_name}"')
    docsearch = PL.from_existing_index(index_name, embedding_function)

In [7]:
llm = Ollama(
    model="llama3.1",
    base_url="http://localhost:11434",
    verbose=True,
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
logging.info(f"using {device}")

2024-09-11 10:11:29.371 | INFO     | __main__:<module>:7 - using cuda


In [10]:
def inference(question: str):

    template = """
    You are a Russian-speaking news correspondent. People will ask you questions about the news.
    Use the following pieces of context to answer the question at the end. Please follow the following rules:
    1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
    2. If you find the answer, write the answer in a concise way with five sentences maximum.
    3. Give the answer on Russian.
    4. Keep a formal communication style

    Context: {context}
    Question: {question}
    Answer:
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

    conversation = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
        return_source_documents=True,
        verbose=True,
        combine_docs_chain_kwargs={"prompt": prompt}
    )

    chat_history = []
    response = conversation({"question": question, "chat_history": chat_history})

    return response, chat_history


In [13]:
question = "Расскажи новость про пожар в Санкт-Петербурге"

response, _ = inference(question)
response['answer']



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    You are a Russian-speaking news correspondent. People will ask you questions about the news.
    Use the following pieces of context to answer the question at the end. Please follow the following rules:
    1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
    2. If you find the answer, write the answer in a concise way with five sentences maximum.
    3. Give the answer on Russian.
    4. Keep a formal communication style

    Context: Около двух часов ночи с понедельника на вторник в жилой квартире на первом этаже дома 117 корпус 1 по Московскому проспекту в Петербурге возник пожар. Как сообщили РИА "Новости" со ссылкой на пресс-службу Главного управления государственной противопожарной службы МВД РФ, находившиеся в квартире двое мужчин и три женщины погибли, скорее всего, 

'Здесь новость о пожаре в Санкт-Петербурге:\n\nВо вторник утром на Московском проспекте в Санкт-Петербурге произошел пожар в жилой квартире. Погибли двое мужчин и три женщины, скорее всего, из-за отравления дымом. Огонь захватил площадь 20 квадратных метров в кухне и коридоре. Тушением занимались пожарные.'

In [57]:
response['source_documents']

[Document(metadata={'date': '1999/09/21', 'tags': 'Все', 'title': 'Пять человек погибли при пожаре в\xa0Петербурге', 'topic': 'Россия', 'url': 'https://lenta.ru/news/1999/09/21/spb_fire/'}, page_content='Около двух часов ночи с понедельника на вторник в жилой квартире на первом этаже дома 117 корпус 1 по Московскому проспекту в Петербурге возник пожар. Как сообщили РИА "Новости" со ссылкой на пресс-службу Главного управления государственной противопожарной службы МВД РФ, находившиеся в квартире двое мужчин и три женщины погибли, скорее всего, из-за отравления дымом. Огонь захватил площадь в 20 квадратных метров в кухне и коридоре. По информации ИТАР-ТАСС, тушением занимались 2 пожарных расчета. Через час огонь был локализован. На место прибыли три бригады "скорой помощи". Из горящей квартиры удалось выбраться только ее хозяйке. Причины пожара пока не установлены, отметили в пресс-службе МВД. По предварительным результатам расследования, пожар произошел из-за неосторожного обращения с о

In [ ]:
torch.cuda.empty_cache()